In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/kpdlhlv/merged_all.txt
/kaggle/input/kpdlhlv/longchau_text_done.txt
/kaggle/input/kpdlhlv/vinmec_diseases_final.jsonl
/kaggle/input/kpdlhlv/drug_tviet_13k600_processed.jsonl
/kaggle/input/kpdlhlv/my_test_final.xlsx
/kaggle/input/kpdlhlv/medlatec_processed_chunks.txt
/kaggle/input/kpdlhlv/drug_tviet_21k899_processed.jsonl
/kaggle/input/kpdlhlv/nhs_translated_formatted_done.txt
/kaggle/input/kpdlhlv/drug_tviet_13k600_processed_done.txt
/kaggle/input/kpdlhlv/youmed_rag_data_done.txt
/kaggle/input/kpdlhlv/nhs_conditions.jsonl
/kaggle/input/kpdlhlv/pharmacity_diseases_merged_done.txt
/kaggle/input/kpdlhlv/vinmec_rag_file_merged.jsonl
/kaggle/input/kpdlhlv/pubmedqa_train_translated.csv
/kaggle/input/kpdlhlv/Test_sample.v1.0_formatted.txt
/kaggle/input/kpdlhlv/vinmec_rag_file_merged_done.txt
/kaggle/input/kpdlhlv/drug_tviet_21k899_processed2_done.txt
/kaggle/input/kpdlhlv/VMB_YN.csv
/kaggle/input/kpdlhlv/vinmec_diseases_final_done.txt
/kaggle/input/kpdlhlv/icd10_rich_vietnam_don

In [2]:
!pip install transformers sentence-transformers faiss-cpu torch  --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is

In [3]:
!pip install langchain langchain-community --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.8/450.8 kB 34.6 MB/s eta 0:00:00


In [4]:
!pip install rank_bm25

In [5]:
import os
import json
import numpy as np
import torch
import faiss
import re
from typing import List, Tuple, Dict
from pathlib import Path

from sentence_transformers import SentenceTransformer, CrossEncoder
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sklearn.feature_extraction.text import TfidfVectorizer

2025-12-15 05:40:56.506430: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765777256.675085      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765777256.721995      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [6]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from huggingface_hub import login

login("HG_KEY")  

# Expansion

In [8]:
def extract_medical_entities(query: str) -> List[str]:
    """Trích xuất các thực thể y tế từ query"""
    entities = []
    
    # ICD code
    icd_pattern = r'\b[A-Z]\d{2}(?:\.\d{1,2})?\b'
    entities.extend(re.findall(icd_pattern, query))
    
    # Số liệu y tế
    numeric_pattern = r'\b\d+(?:[./]\d+)?(?:\s*%|°C|mg|ml|mmHg)?\b'
    entities.extend(re.findall(numeric_pattern, query))
    
    return entities

def preprocess_medical_query(query: str) -> str:
    """Tiền xử lý query"""
    query = re.sub(r'\s+', ' ', query).strip()
    query = query.replace('...', '.')
    query = re.sub(r'([?.!])\1+', r'\1', query)
    query = re.sub(r'(\d+)\s*/\s*(\d+)', r'\1/\2', query)
    return query

def boost_query_with_entities(query: str, entities: List[str]) -> str:
    """Tăng trọng số cho entities quan trọng"""
    if not entities:
        return query
    entity_boost = " ".join(entities)
    return f"{query} {entity_boost}"

def full_query_preprocessing(query: str) -> str:
    """Pipeline đầy đủ xử lý query"""
    query = preprocess_medical_query(query)
    entities = extract_medical_entities(query)
    query = boost_query_with_entities(query, entities)

    return query


# Chunking (chia nhỏ tài liệu)

In [9]:
# ============ Tách dữ liệu thành từng chunk ======================
# Xem xét tách theo section
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

def chunk_medical_docs(docs, chunk_size=400, chunk_overlap=200):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", ".", " ", ""],
    )

    chunks = splitter.split_text(docs)
    return chunks

# Embedding các chunk

In [10]:
from sentence_transformers import SentenceTransformer

def load_embedding_model():
    return SentenceTransformer("intfloat/multilingual-e5-large")

def embed_chunks(embed_model, chunks):
    # Thêm prefix theo hướng dẫn của model E5
    chunks_with_prefix = [f"passage: {c}" for c in chunks] 
    
    vectors = embed_model.encode(
        chunks_with_prefix, # Encode đoạn văn có prefix
        normalize_embeddings=True,
        show_progress_bar=True,
        batch_size=16,
        device='cuda'
    )
    return chunks, vectors # Trả về chunks gốc (không prefix) để hiển thị

# Tạo index faiss để tìm kiếm nhanh

In [11]:
import faiss

def build_faiss_index(vectors):
    dim = vectors.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(vectors)
    return index


# Search hybrid kết hợp tìm kiếm từ khóa  

In [12]:
def search_hybrid_rrf_fast(
    embed_model, 
    faiss_index, 
    keyword_model,   # TfidfVectorizer
    keyword_matrix,  # Ma trận TF-IDF
    doc_texts, 
    query, 
    k: int = 10,
    rrf_k: int = 60,
    use_preprocessing: bool = True
) -> list[int]:
    """
    Hybrid Search tốc độ cao: Kết hợp Vector (GPU) và TF-IDF (CPU C++) bằng RRF
    """
    
    # 1. Xử lý query
    if use_preprocessing:
        query_processed = full_query_preprocessing(query)
    else:
        query_processed = query
        
    # ============ DENSE SEARCH (VECTOR) ============
    # Thêm prefix "query: " cho câu hỏi (Bắt buộc với E5)
    query_for_embed = f"query: {query_processed}"
    query_vec = embed_model.encode(
        [query_for_embed], 
        normalize_embeddings=True, 
        show_progress_bar=False
    )
    # Lấy top k*2 từ vector
    dense_distances, dense_indices = faiss_index.search(query_vec, k * 2)
    
    # ============ SPARSE SEARCH (TF-IDF) ============
    # Tận dụng tốc độ của sklearn để tính toán
    query_tfidf = keyword_model.transform([query_processed])
    
    # Tính điểm tương đồng (Dot product)
    sparse_scores_raw = (query_tfidf @ keyword_matrix.T).toarray()[0]
    
    # Lấy Top k*2 index có điểm cao nhất (dùng argpartition cho nhanh)
    search_k_sparse = min(k * 2, len(doc_texts))
    # Lấy index (chưa sort)
    top_sparse_indices_unsorted = np.argpartition(sparse_scores_raw, -search_k_sparse)[-search_k_sparse:]
    # Sort lại nhóm top này
    top_sparse_indices = top_sparse_indices_unsorted[np.argsort(sparse_scores_raw[top_sparse_indices_unsorted])][::-1]
    
    # ============ TÍNH ĐIỂM RRF (RECIPROCAL RANK FUSION) ============
    rrf_score_map = {}

    # 1. Cộng điểm rank cho Vector
    for rank, idx in enumerate(dense_indices[0]):
        if idx not in rrf_score_map: rrf_score_map[idx] = 0
        rrf_score_map[idx] += 1 / (rank + 1 + rrf_k)
        
    # 2. Cộng điểm rank cho TF-IDF
    for rank, idx in enumerate(top_sparse_indices):
        # Chỉ tính nếu có điểm > 0 (có từ khóa xuất hiện)
        if sparse_scores_raw[idx] > 0:
            if idx not in rrf_score_map: rrf_score_map[idx] = 0
            rrf_score_map[idx] += 1 / (rank + 1 + rrf_k)
    
    # Sắp xếp kết quả cuối cùng theo điểm RRF
    sorted_results = sorted(rrf_score_map.items(), key=lambda x: x[1], reverse=True)
    
    # Trả về danh sách index
    return [idx for idx, score in sorted_results[:k]]

# Rerank tối ưu retriver bằng BGE-reranker

In [13]:
from sentence_transformers import CrossEncoder
import torch

def load_reranker():
    # Ưu tiên GPU nếu có
    device = "cuda" if torch.cuda.is_available() else "cpu"

    try:
        print("🔎 Đang load BGE reranker v2 m3 (model lớn)...")
        return CrossEncoder(
            "BAAI/bge-reranker-v2-m3",
            device=device
        )
    except Exception as e:
        print("⚠️ Không load được BGE v2 m3 → chuyển sang MiniLM. Lỗi:", e)
        print("🔁 Load MiniLM reranker nhẹ (GPU nếu có)...")

        # Model này nhẹ (~80MB), chạy tốt trên GPU Kaggle
        return CrossEncoder(
            "cross-encoder/ms-marco-MiniLM-L-12-v2",
            device=device
        )
        
def rerank_chunks(reranker, query, chunk_texts, top_k=3):
    # Chuẩn bị batch pairs
    pairs = [[query, t] for t in chunk_texts]

    # Predict batch → GPU tối ưu
    scores = reranker.predict(pairs)

    # Sắp xếp từ cao xuống thấp
    best_idx = scores.argsort()[::-1][:top_k]

    return [chunk_texts[i] for i in best_idx]

# Generate answers

In [14]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import re 

def load_slm(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        torch_dtype=torch.float16
    )
    return tokenizer, model


def clean_answer(text: str) -> str:
    text = text.strip().lower()
    
    # Tìm tất cả vị trí xuất hiện của "đúng" và "sai" (word boundary)
    true_positions = [m.start() for m in re.finditer(r"\bđúng\b", text)]
    false_positions = [m.start() for m in re.finditer(r"\bsai\b", text)]
    
    # Nếu text quá dài (> 50 ký tự) -> không tuân thủ format
    if len(text) > 50:
        # Ưu tiên từ xuất hiện cuối cùng
        if true_positions and false_positions:
            return "Đúng" if true_positions[-1] > false_positions[-1] else "Sai"
        elif true_positions:
            return "Đúng"
        else:
            return "Sai"
    
    # Text ngắn <= 50 ký tự
    if not true_positions and not false_positions:
        return "Sai"
    if true_positions and not false_positions:
        return "Đúng"
    if false_positions and not true_positions:
        return "Sai"
    
    # Cả hai đều có -> lấy từ xuất hiện cuối cùng
    return "Đúng" if true_positions[-1] > false_positions[-1] else "Sai"

def answer_with_qwen(tokenizer, model, query, contexts):
    context_block = "\n\n---\n\n".join(contexts)
    
    # System prompt
    system_prompt = """Bạn là một nhân viên y tế chuyên đánh giá tính đúng/sai của các khẳng định dựa trên thông tin tham khảo.

    QUY TẮC:
    - CHỈ trả lời MỘT trong hai từ: "Đúng" HOẶC "Sai"
    - KHÔNG viết thêm BẤT KỲ từ nào khác
    - KHÔNG giải thích, KHÔNG thêm dấu câu
    - Nếu thông tin tham khảo không đủ bằng chứng rõ ràng → trả lời "Sai"
    - Nếu thông tin tham khảo không đề cập đến khẳng định → trả lời "Sai"
    - Dựa vào thông tin tham khảo trước, nếu không có thông tin thì dùng kiến thức bên ngoài
    - Đọc kỹ Văn bản tham khảo từ đầu đến cuối."""

    # User prompt
    user_prompt = f"""[THÔNG TIN THAM KHẢO]
    {context_block}
    
    [KHẲNG ĐỊNH CẦN ĐÁNH GIÁ]
    {query}
    
    [TRẢ LỜI]
    """

    # Tạo messages theo format chat
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    
    # Apply chat template
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    inputs = tokenizer([text], return_tensors="pt").to(model.device)

    output = model.generate(
        **inputs,
        max_new_tokens=10,
        temperature=0.1,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )
    
    generated_ids = output[0][inputs["input_ids"].shape[1]:]
    raw_answer = tokenizer.decode(generated_ids, skip_special_tokens=True)
    
    #print("RAW ANSWER:", repr(raw_answer))
    return clean_answer(raw_answer)


# Toàn bộ pipeline

In [15]:

# =========================================================
# 1. LOAD DỮ LIỆU & CHUNKING
# =========================================================
print("1. Đang load dữ liệu...")
data_paths = ["/kaggle/input/kpdlhlv/medlatec_diseases_clean_done.txt",
              "/kaggle/input/kpdlhlv/nhs_translated_formatted_done.txt",
              "/kaggle/input/kpdlhlv/pharmacity_diseases_merged_done.txt",
              "/kaggle/input/kpdlhlv/vinmec_diseases_final_done.txt",
              "/kaggle/input/kpdlhlv/youmed_rag_data_done.txt",
              "/kaggle/input/kpdlhlv/longchau_text_done.txt",
              "/kaggle/input/kpdlhlv/icd10_rich_vietnam_done.txt",
              "/kaggle/input/kpdlhlv/drug_tviet_21k899_processed2_done.txt",
              "/kaggle/input/kpdlhlv/drug_tviet_13k600_processed_done.txt"
             ]
raw_text = ""

for p in data_paths:
    path = Path(p)
    if path.exists():
        with open(path, "r", encoding="utf-8") as f:
            raw_text += f.read().strip() + "\n"
    else:
        print(f"File không tồn tại: {p}")

print(f"   -> Độ dài text: {len(raw_text)}")

# Chunking
chunks = chunk_medical_docs(raw_text)

# =========================================================
# 2. ĐÓNG BĂNG DỮ LIỆU & TẠO INDEX 
# =========================================================
print("2. Đang đóng băng dữ liệu & tạo Index (Dùng hàm helper)...")

# Tạo list cố định để đảm bảo index không bị lệch
fixed_chunk_texts = list(chunks)
print(f"   Tổng số chunks: {len(fixed_chunk_texts)}")

# [PHẦN A] Giữ nguyên đoạn tạo TF-IDF cũ
print("Đang tạo Index TF-IDF...")
keyword_model = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b", max_features=200000)
keyword_matrix = keyword_model.fit_transform(fixed_chunk_texts)
print("Xong TF-IDF.")


# B. Tạo Index Vector (FAISS) 
print("Đang tạo Embedding & FAISS (Gọi hàm load_embedding_model, embed_chunks, build_faiss_index)...")
# 1. Load Model
embed_model = load_embedding_model() # <- Dùng hàm của bạn

# 2. Embed (Quan trọng: truyền fixed_chunk_texts vào)
# Bỏ qua giá trị chunk_texts trả về vì nó chính là fixed_chunk_texts
_, vectors = embed_chunks(embed_model, fixed_chunk_texts) 

# 3. Build Index
index = build_faiss_index(vectors) # <- Dùng hàm của bạn
print("Xong Indexing.")


# =========================================================
# 3. LOAD CÁC MODEL KHÁC (RERANKER & LLM)
# =========================================================
print("3. Đang load Reranker & LLM...")
reranker = load_reranker()
tokenizer, model = load_slm("google/gemma-3-1b-it") 

1. Đang load dữ liệu...
   -> Độ dài text: 39763464
2. Đang đóng băng dữ liệu & tạo Index (Dùng hàm helper)...
   Tổng số chunks: 173019
Đang tạo Index TF-IDF...
Xong TF-IDF.
Đang tạo Embedding & FAISS (Gọi hàm load_embedding_model, embed_chunks, build_faiss_index)...


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

# Test của thầy

In [16]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd

# 1. Load dữ liệu kiểm tra
test_path = "/kaggle/input/kpdlhlv/Test_sample.v1.0.xlsx"
df = pd.read_excel(test_path)

# Các list lưu kết quả
y_true = []
y_pred = []
list_questions = []
list_answers_raw = []
list_contexts = []

# Hàm chuẩn hóa nhãn (Đúng/Sai)
def norm_label(x):
    if x is None:
        return None
    x = str(x).strip().lower()
    if x in ["đúng", "true", "1", "yes", "y", "correct"]:
        return "Đúng"
    else:
        return "Sai"

print(f"Bắt đầu đánh giá trên {len(df)} câu hỏi...")

# 2. Vòng lặp đánh giá
for i, row in df.iterrows():
    query = row["Mệnh đề Câu hỏi (VIETNAMESE TEXT ONLY)"]
    gt = norm_label(row["Đáp án (TRUE/FALSE)"])

    try:
        idx_list = search_hybrid_rrf_fast(
            embed_model, 
            index, 
            keyword_model,   # Truyền model
            keyword_matrix,  # Truyền ma trận
            fixed_chunk_texts, 
            query, 
            k=10,
            rrf_k=60,
            use_preprocessing=True
        )
        
        # Lấy nội dung text từ index (Dùng fixed_chunk_texts để tránh lệch)
        retrieved = [fixed_chunk_texts[j] for j in idx_list]
        
    except Exception as e:
        print(f"Lỗi ở câu {i}: {e}")
        retrieved = []
    # -----------------------------

    # Rerank: Lọc lại để lấy top 3 context tốt nhất
    final_contexts = rerank_chunks(reranker, query, retrieved, top_k=3)

    # Generate: Hỏi mô hình
    ans = answer_with_qwen(tokenizer, model, query, final_contexts)
    pred = norm_label(ans)

    # Lưu kết quả
    y_true.append(gt)
    y_pred.append(pred)
    list_questions.append(query)
    list_answers_raw.append(ans)
    list_contexts.append("\n---\n".join(final_contexts))

    # In tiến độ (cứ 10 câu in 1 lần cho đỡ rối)
    if (i + 1) % 10 == 0:
        print(f"Đã xử lý {i + 1}/{len(df)} câu.")

# === 3. Tính Metrics ===
# Loại bỏ các trường hợp None nếu có (để tránh lỗi metric)
valid_indices = [i for i, (yt, yp) in enumerate(zip(y_true, y_pred)) if yt is not None and yp is not None]
y_true_clean = [y_true[i] for i in valid_indices]
y_pred_clean = [y_pred[i] for i in valid_indices]

acc  = accuracy_score(y_true_clean, y_pred_clean)
prec = precision_score(y_true_clean, y_pred_clean, pos_label="Đúng", zero_division=0)
rec  = recall_score(y_true_clean, y_pred_clean, pos_label="Đúng", zero_division=0)
f1   = f1_score(y_true_clean, y_pred_clean, pos_label="Đúng", zero_division=0)

print("\n" + "="*30)
print("🔎 KẾT QUẢ ĐÁNH GIÁ CUỐI CÙNG:")
print("="*30)
print(f"Accuracy : {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall   : {rec:.4f}")
print(f"F1-score : {f1:.4f}")

# === 4. Xuất CSV ===
result_df = pd.DataFrame({
    "Question": list_questions,
    "GroundTruth": y_true,
    "Prediction": y_pred,
    "Raw Model Output": list_answers_raw,
    "Retrieved Contexts": list_contexts
})

out_path = "/kaggle/working/RAG_eval_output_hybrid.csv"
result_df.to_csv(out_path, index=False, encoding="utf-8-sig")

Bắt đầu đánh giá trên 1246 câu hỏi...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
W1215 06:43:56.517000 20 torch/_inductor/utils.py:1137] [0/0] Not enough SMs to use max_autotune_gemm mode
W1215 06:43:56.547000 20 torch/_inductor/utils.py:1137] [0/0] Not enough SMs to use max_autotune_gemm mode
skipping cudagraphs due to skipping cudagraphs due to multiple devices: device(type='cuda', index=0), device(type='cuda', index=1)
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
skipping cudagraphs due to skipping cudagraphs due to multiple devices: device(type='cuda', index=0), device(type='cuda', index=1)
The following generation flags are not valid and may be ignored: ['t

Đã xử lý 10/1246 câu.


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p'

Đã xử lý 20/1246 câu.


...

Đã xử lý 1230/1246 câu.


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p'

Đã xử lý 1240/1246 câu.


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



🔎 KẾT QUẢ ĐÁNH GIÁ CUỐI CÙNG:
Accuracy : 0.7480
Precision: 0.7210
Recall   : 0.8563
F1-score : 0.7828


# Test của mình

In [17]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd

my_test_path = "/kaggle/input/kpdlhlv/my_test_final.xlsx"
df = pd.read_excel(my_test_path)

# Các list lưu kết quả
y_true = []
y_pred = []
list_questions = []
list_answers_raw = []
list_contexts = []

def norm_label(x):
    if x is None:
        return None
    x = str(x).strip().lower()
    if x in ["đúng", "true", "1", "yes", "y", "correct", "True"]:
        return "Đúng"
    else:
        return "Sai"

print(f"Bắt đầu đánh giá trên {len(df)} câu hỏi...")

for i, row in df.iterrows():
    try:
        query = row["Question"]
        gt = norm_label(row["Label"])

        # ===== RETRIEVAL =====
        idx_list = search_hybrid_rrf_fast(
            embed_model, 
            index, 
            keyword_model,
            keyword_matrix,
            fixed_chunk_texts, 
            query, 
            k=10,
            rrf_k=60,
            use_preprocessing=True
        )
        retrieved = [fixed_chunk_texts[j] for j in idx_list]

        # ===== RERANK (guard empty) =====
        if len(retrieved) == 0:
            final_contexts = []
        else:
            final_contexts = rerank_chunks(reranker, query, retrieved, top_k=3)

        # ===== GENERATION =====
        ans = answer_with_qwen(tokenizer, model, query, final_contexts)
        pred = norm_label(ans)

    except Exception as e:
        # CATCH TOÀN BỘ LỖI → BỎ QUA CÂU NÀY
        print(f"⚠️ Bỏ qua câu {i} do lỗi: {e}")
        continue

    # ===== LƯU KẾT QUẢ =====
    y_true.append(gt)
    y_pred.append(pred)
    list_questions.append(query)
    list_answers_raw.append(ans)
    list_contexts.append("\n---\n".join(final_contexts))

    if (i + 1) % 10 == 0:
        print(f"Đã xử lý {i + 1}/{len(df)} câu.")

# ===== METRICS =====
valid_idx = [i for i,(a,b) in enumerate(zip(y_true,y_pred)) if a is not None and b is not None]
y_true_clean = [y_true[i] for i in valid_idx]
y_pred_clean = [y_pred[i] for i in valid_idx]

acc  = accuracy_score(y_true_clean, y_pred_clean)
prec = precision_score(y_true_clean, y_pred_clean, pos_label="Đúng", zero_division=0)
rec  = recall_score(y_true_clean, y_pred_clean, pos_label="Đúng", zero_division=0)
f1   = f1_score(y_true_clean, y_pred_clean, pos_label="Đúng", zero_division=0)

print("\n==============================")
print("🔎 KẾT QUẢ ĐÁNH GIÁ CUỐI CÙNG")
print("==============================")
print(f"Accuracy : {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall   : {rec:.4f}")
print(f"F1-score : {f1:.4f}")

# ===== EXPORT =====
result_df = pd.DataFrame({
    "Question": list_questions,
    "GroundTruth": y_true,
    "Prediction": y_pred,
    "Raw Model Output": list_answers_raw,
    "Retrieved Contexts": list_contexts
})

out_path = "/kaggle/working/my_test.csv"
result_df.to_csv(out_path, index=False, encoding="utf-8-sig")


Bắt đầu đánh giá trên 1331 câu hỏi...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p'

Đã xử lý 10/1331 câu.


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p'

Đã xử lý 20/1331 câu.


...

Đã xử lý 1320/1331 câu.


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p'

Đã xử lý 1330/1331 câu.


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



🔎 KẾT QUẢ ĐÁNH GIÁ CUỐI CÙNG
Accuracy : 0.7806
Precision: 0.7668
Recall   : 0.7668
F1-score : 0.7668
